In [6]:
import numpy as np
import nltk
import sklearn
import operator
import requests
import pandas as pd 
import numpy as np
import pandas as pd
import nltk
import sklearn
import operator
import requests
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
import re
from collections import Counter
from nltk.util import ngrams
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
lemmatizer = nltk.stem.WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('stopwords') # If needed
nltk.download('punkt') # If needed
nltk.download('wordnet') # If needed

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\khaya\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\khaya\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\khaya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\khaya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\khaya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
#Positive and negative reviews

#train data
path_train_pos='imdb_train_pos.txt'
train_pos=open(path_train_pos,encoding='utf8').readlines()
path_train_neg='imdb_train_neg.txt'
train_neg=open(path_train_neg,encoding='utf8').readlines()

#test data
path_test_pos='imdb_test_pos.txt'
test_pos=open(path_test_pos,encoding='utf8').readlines()
path_test_neg='imdb_test_neg.txt'
test_neg=open(path_test_neg,encoding='utf8').readlines()

#dev data
path_dev_pos='imdb_dev_pos.txt'
dev_pos=open(path_dev_pos,encoding='utf8').readlines()
path_dev_neg='imdb_dev_neg.txt'
dev_neg=open(path_dev_neg,encoding='utf8').readlines()

In [5]:
train=[]
for pos_review in train_pos:
    train.append((pos_review,1))
for neg_review in train_neg:
    train.append((neg_review,0))


test=[]
for pos_review in test_pos:
    test.append((pos_review,1))
for neg_review in test_neg:
    test.append((neg_review,0))


dev=[]
for pos_review in dev_pos:
    dev.append((pos_review,1))
for neg_review in dev_neg:
    dev.append((neg_review,0))

In [20]:
    def clean_doc(doc):
    # Split string into list
    tokens = doc.split()
    # Get rid of unwanted punctuation
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    #Use isalpha() to remove words that don't begin with a letter from the alphabet
    tokens = [word for word in tokens if word.isalpha()]
    # Stop_words removes mispelled words, punctuation and symbols
    stop_words=set(nltk.corpus.stopwords.words('english'))
    stop_words.add(".")
    stop_words.add(",")
    stop_words.add("--")
    stop_words.add("``")
    stop_words.add("#")
    stop_words.add("@")
    stop_words.add(":")
    stop_words.add("'s")
    stop_words.add("’")
    stop_words.add("...")
    stop_words.add("n't")
    stop_words.add("'")
    stop_words.add("-")
    stop_words.add(";")
    stop_words.add("http")
    stop_words.add("!")
    stop_words.add("?")
    stop_words.add("&")
    stop_words.add(")")
    stop_words.add('wa')
    stop_words.add('u')
    stop_words.add("'re")
    stop_words.add("''")
    stop_words.add('(')
    stop_words.add('ca')
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    list_tokens=[]
    for token in tokens:
        list_tokens.append(lemmatizer.lemmatize(token).lower())
    #Assign tag to token list to show what it is
    test = nltk.pos_tag(list_tokens)
    #Only use adjectives and verbs
    final_tokens = [a[0] for a in test if (a[1] == 'JJ' or a[1] == 'VERB')]
    return final_tokens
 
# load doc and add to vocab
def add(filename, vocab):
    
    tokens = clean_doc(filename)
    # update counts
    vocab.update(tokens)
 
 
#Vocab
vocab = Counter()
# add all docs to vocab
for instance in ([i[0] for i in train]):
    add(instance, vocab)

# Vocab size
print(len(vocab))
# Top 50 words
print(vocab.most_common(50))

29802
[('good', 8100), ('great', 5386), ('bad', 5340), ('many', 4127), ('much', 4088), ('little', 3368), ('first', 2927), ('real', 2718), ('old', 2334), ('new', 2268), ('funny', 2089), ('big', 2028), ('young', 1951), ('whole', 1748), ('br', 1745), ('original', 1706), ('ive', 1683), ('last', 1680), ('u', 1543), ('main', 1414), ('different', 1349), ('sure', 1310), ('american', 1289), ('special', 1286), ('dont', 1275), ('true', 1239), ('black', 1232), ('hard', 1208), ('second', 1169), ('high', 1159), ('short', 1151), ('poor', 1146), ('cant', 1104), ('classic', 1076), ('give', 1066), ('excellent', 1039), ('beautiful', 1029), ('right', 1028), ('full', 1018), ('human', 978), ('nice', 972), ('stupid', 965), ('interesting', 960), ('dead', 941), ('terrible', 925), ('wrong', 920), ('im', 918), ('enough', 900), ('long', 890), ('small', 888)]


In [27]:
vectorizer = TfidfVectorizer(clean_doc,ngram_range=(1,2),max_features=1000)
matrix = vectorizer.fit_transform([i[0] for i in train])

In [28]:
#Train SVM model
xtrain = matrix.toarray()
ytrain = [i[1] for i in train]
svm_clf = sklearn.svm.SVC(kernel="linear",gamma='auto')
svm_clf.fit(np.asarray(xtrain),np.asarray(ytrain))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [29]:
xdev = vectorizer.transform([i[0] for i in dev]).toarray()
predictions = svm_clf.predict(xdev)
print(sklearn.metrics.classification_report(predictions,[i[1] for i in dev]))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85      2442
           1       0.86      0.85      0.85      2558

    accuracy                           0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000



In [31]:
vectorizer2 = TfidfVectorizer(clean_doc,ngram_range=(1,3),max_features=2500)
matrix = vectorizer2.fit_transform(np.asarray([i[0] for i in train]))
xtrain = matrix.toarray()
ytrain=[i[1] for i in train]

svm_clf_nu=sklearn.svm.SVC(kernel="linear",gamma='auto')
svm_clf_nu.fit(np.asarray(xtrain),np.asarray(ytrain))

xdev = vectorizer2.transform([i[0] for i in dev]).toarray()
predictions = svm_clf_nu.predict(xdev)
print(sklearn.metrics.classification_report(predictions,[i[1] for i in dev]))


              precision    recall  f1-score   support

           0       0.86      0.87      0.87      2452
           1       0.88      0.87      0.87      2548

    accuracy                           0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000



In [32]:
vectorizer_f = TfidfVectorizer(clean_doc,ngram_range=(1,2),max_features=5000)
xtrain = vectorizer_f.fit_transform([i[0] for i in train]).toarray()
ytrain = np.asarray([i[1] for i in train])

xFeatureBest = SelectKBest(chi2, k=500).fit(xtrain, ytrain)
xtrainFeatureBest = xFeatureBest.transform(xtrain)

xtest = vectorizer_f.transform([i[0] for i in test]).toarray()
xtestFeatureBest =  xFeatureBest.transform(xtest)

ytest = np.asarray([i[1] for i in test])

In [34]:
svm_clf_3=sklearn.svm.SVC(kernel="linear",gamma='auto')
svm_clf_3.fit(np.asarray(xtrainFeatureBest),ytrain)
predFeatureBest = svm_clf_3.predict(xtestFeatureBest)
print(sklearn.metrics.classification_report(predFeatureBest,ytest))

              precision    recall  f1-score   support

           0       0.84      0.89      0.86      2375
           1       0.89      0.85      0.87      2625

    accuracy                           0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000



In [36]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
ytestGold = ytest
ytextPred = predFeatureBest
precision=precision_score(ytestGold, ytextPred, average='macro')
recall=recall_score(ytestGold, ytextPred, average='macro')
f1=f1_score(ytestGold, ytextPred, average='macro')
accuracy=accuracy_score(ytestGold, ytextPred)

print ("Precision: "+str(round(precision,3)))
print ("Recall: "+str(round(recall,3)))
print ("F1-Score: "+str(round(f1,3)))
print ("Accuracy: "+str(round(accuracy,3)))

Precision: 0.869
Recall: 0.868
F1-Score: 0.868
Accuracy: 0.868
